In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_csv('device_network_data.csv')
df

,Epoch Time,Value
0,1704042000000,1
1,1704042026000,1
2,1704042035000,1
3,1704042037000,1
4,1704042059000,1
...,...,...
7214,1704128340000,1
7215,1704128361000,1
7216,1704128378000,Null
7217,1704128391000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Epoch Time  7219 non-null   int64 
 1   Value       7216 non-null   object
dtypes: int64(1), object(1)
memory usage: 112.9+ KB


After inspection above, we can get following informations:
1. Total number of data points is 7219
2. There are two columns: 'Epoch Time' (integer) and 'Value' (object/string)
3. There are missing values in 'Value' column, at least 3 missing values. We will inspect this further in the next step

Based on that information, we will do the following:
1. Inspect the missing values in 'Value' column. Are there any other kind besides "Null" or "NaN"
2. Convert 'Value' column to integer
3. Convert 'Epoch Time' column to datetime

After that, we will begin counting the total downtime in the dataset.

In [5]:
df.Value.unique()

array(['1', '0', 'Null', nan], dtype=object)

We can see that 'Value' column has 'Null' and nan values. We will inspect them further

In [6]:
df.Value.value_counts()

Value
1       4941
0       2268
Null       7
Name: count, dtype: int64

In [7]:
df[df.Value == 'Null']

,Epoch Time,Value
22,1704042239000,Null
23,1704042240000,Null
24,1704042264000,Null
445,1704046759000,Null
446,1704046775000,Null
5160,1704102803000,Null
7216,1704128378000,Null


In [8]:
df[df.Value.isna()]

,Epoch Time,Value
25,1704042267000,NaN
31,1704042336000,NaN
32,1704042337000,NaN


Now we know that our data has 10 missing values in 'Value' column. We will fill this missing values with the last non-null value before itself. 

In [9]:
df.loc[:, "Value"] = df.Value.replace("Null", np.nan)
df.loc[:, "Value"] = df.Value.ffill()

In [10]:
df.Value.value_counts()

Value
1    4945
0    2274
Name: count, dtype: int64

In [11]:
print(f'total data points: {df.Value.value_counts().sum()}')
print(f'Unique data points: {df.Value.unique()}')

total data points: 7219
Unique data points: ['1' '0']


In [12]:
df.Value.isna().sum()

0

Now we see that the total data points match with our first inspection. We will continue to convert 'Value' column to integer and 'Epoch Time' column to datetime

In [13]:
df["Value"] = df.Value.astype(int)
df["date_time"] = df.loc[:, "Epoch Time"].apply(lambda x: datetime.fromtimestamp(x / 1e3))

df.head(10)

,Epoch Time,Value,date_time
0,1704042000000,1,2024-01-01 00:00:00
1,1704042026000,1,2024-01-01 00:00:26
2,1704042035000,1,2024-01-01 00:00:35
3,1704042037000,1,2024-01-01 00:00:37
4,1704042059000,1,2024-01-01 00:00:59
5,1704042060000,1,2024-01-01 00:01:00
6,1704042070000,1,2024-01-01 00:01:10
7,1704042086000,0,2024-01-01 00:01:26
8,1704042097000,0,2024-01-01 00:01:37
9,1704042098000,0,2024-01-01 00:01:38


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Epoch Time  7219 non-null   int64         
 1   Value       7219 non-null   int32         
 2   date_time   7219 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int32(1), int64(1)
memory usage: 141.1 KB


In [20]:
def count_downtime(df):
    i, total_down = 0, 0
    while i in range(df.shape[0] - 1):
        if df.Value[i] == 1:
            total_down += (df.date_time[i + 1] - df.date_time[i]).seconds
        i += 1

    return total_down

In [27]:
# Check the function with the test data shown in github repo
data_test_1 = [
    (1704042000000, 0),
    (1704042026000, 1),
    (1704042035000, 1),
    (1704042037000, 0),
    (1704042059000, 1),
    (1704042060000, 1)
]

test1 = pd.DataFrame(data_test_1, columns=["epoch_time", "Value"])
test1["date_time"] = test1["epoch_time"].apply(lambda x: datetime.fromtimestamp(x / 1e3))

print(f'Total Downtime: {count_downtime(test1)} seconds')

Total Downtime: 12 seconds


In [29]:
# Check the function with the test data shown in github repo
data_test_2 = [
    (1704042000000, 1),
    (1704042026000, 1),
    (1704042035000, 1),
    (1704042037000, 1),
    (1704042059000, 1),
]

test1 = pd.DataFrame(data_test_2, columns=["epoch_time", "Value"])
test1["date_time"] = test1["epoch_time"].apply(lambda x: datetime.fromtimestamp(x / 1e3))

print(f'Total Downtime: {count_downtime(test1)} seconds')

Total Downtime: 59 seconds


In [43]:
from time import gmtime, strftime

# Count the downtime for the all of data
downtime = count_downtime(df)
print(f'Total Downtime: {downtime} seconds')
print(f'or: {strftime("%H hour, %M minutes, and %S seconds", gmtime(downtime))}')

Total Downtime: 58310 seconds
or: 16 hour, 11 minutes, and 50 seconds
